In [4]:
import numpy as np
import sys

In [5]:
with open('content\sherlock-holmes.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))

chars = sorted(list(set(text)))
print('total chars: ', len(chars))



text length 610871
total chars:  59


In [7]:
bad_chars = [';','½','£', 'à', 'â', 'è', 'é']
for i in range(len(bad_chars)):
  text = text.replace(bad_chars[i],"")
chars = sorted(list(set(text)))
textlen = len(text)
charlen = len(chars)
print("length is " + str(textlen))
print("length is " + str(charlen))
print(chars)

length is 610619
length is 52
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [2]:
#Model building
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint

In [11]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# optimizer = RMSprop(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=adam)

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [13]:
import random
from tensorflow.keras.callbacks import LambdaCallback
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# def on_epoch_end(epoch, logs):
#     print()
#     print('----- Generating text after Epoch: %d' % epoch)

#     start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0]:
#         print('----- diversity:', diversity)

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(400):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()
# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "weights3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [15]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [checkpoint, reduce_lr]

In [16]:
model.fit(x, y, batch_size=128, epochs=20, callbacks=callbacks)

Epoch 1/4
1591/1591 [==============================] - ETA: 0s - loss: 2.2259
Epoch 1: loss improved from inf to 2.22594, saving model to weights3.hdf5
1591/1591 [==============================] - 196s 122ms/step - loss: 2.2259 - lr: 0.0010
Epoch 2/4
1590/1591 [============================>.] - ETA: 0s - loss: 1.8070
Epoch 2: loss improved from 2.22594 to 1.80697, saving model to weights3.hdf5
1591/1591 [==============================] - 205s 129ms/step - loss: 1.8070 - lr: 0.0010
Epoch 3/4
1591/1591 [==============================] - ETA: 0s - loss: 1.6499
Epoch 3: loss improved from 1.80697 to 1.64987, saving model to weights3.hdf5
1591/1591 [==============================] - 229s 144ms/step - loss: 1.6499 - lr: 0.0010
Epoch 4/4
1590/1591 [============================>.] - ETA: 0s - loss: 1.5473
Epoch 4: loss improved from 1.64987 to 1.54733, saving model to weights3.hdf5
1591/1591 [==============================] - 223s 140ms/step - loss: 1.5473 - lr: 0.0010


In [17]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [19]:
filename = 'weights3.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer ='adam')

In [20]:
print(generate_text(500, 0.3))

his arm. "there are a few of us who frequed to he was a took of the strook of the street of the street of the seet of a compan to the prase come be an the dears was should the seet that i have to the cort and of the seature was stard to the seet of the street of his fact. there is no have all the good to an one the seed of the street of all the bellor stard up and the port of the
     ous a glown the sent of the sitter of the little street of him for the
     seed the strook and the the string of his gaid of his come to the she to the
